# Problem 6

更大规模的电路, 尝试去除噪声. 什么方法都可以, ZNE 也可以

In [1]:
import tensorcircuit as tc
from tensorcircuit.cloud import apis
from tensorcircuit.noisemodel import NoiseConf
from tensorcircuit.noisemodel import circuit_with_noise
import numpy as np


# Requirement: The error mitigation results should be as close as possible to the noiseless results while using as few shots as possible.
qubit_num = 4
depth_num = 3
global sample_total
sample_total = 0
sim_backend = "expect" # OR "sample"
# In the example, we use a density matrix simulator to directly calculate the exact value of the observable's expectation. 
# However, on a real quantum computer, we can only obtain results from probability sampling. 
# Therefore, if we want to simulate a real quantum computer as closely as possible, sim_backend be set to "sample" here, 


# In this task, you can only use the following function to simulate circuit sampling. This simulator will automatically add noise.
def sampling_circuit(circ,sample_num = 10000,scale_factor = 1):

    if scale_factor <= 0.99:
        print("scale factor is too small! We will set scale factor to 1")
        scale_factor = 1

    if 0.01*scale_factor/8 >= 1/16:
        print("error rate is too high! We will set scale factor to 50")
        scale_factor = 50


    global sample_total
    sample_total += sample_num


    noise_conf = NoiseConf()
    error1 = tc.channels.generaldepolarizingchannel(0.01*scale_factor/8, 2)
    noise_conf.add_noise("cz", error1) # Rule for adding noise: Add a noise channel like error1 after each CZ gate
    
    circ2 = circuit_with_noise(circ, noise_conf)
    if sim_backend == "sample":
        return circ2.sample(sample_num,format = "count_dict_bin",staticmethod = True)
    else:
        return np.real(circ2.expectation_ps(z=[0]))



    

def count_dict_bin2expect(result0):
    if sim_backend == "sample":
        S = 0
        shot_num = 0
        for value in result0.values():
            shot_num += value

        for key in result0:
            if key[0] == '0':
                S += result0[key]
            else:
                S -= result0[key]
        S = S/shot_num
        return S
    else:
        return np.real(result0)


    



# circuit example
def QGAN_circuit(params,depth_num = 2):
    # depth_num indicates the number of repetitions of the basic layer in this circuit
    # Each basic layer contains an RY operation on each qubit and an entanglement layer composed of CZ gates

    circ = tc.DMCircuit(qubit_num)
    for index in range(0,depth_num):
        for jndex in range(qubit_num):

            circ.ry(jndex, theta=params[index*qubit_num+jndex])
            
        for jndex in range(qubit_num-1):
            circ.cz(jndex,jndex+1)

        
    
    return circ

param0 = np.random.rand(qubit_num*depth_num)*2*np.pi
circ = QGAN_circuit(param0,depth_num =depth_num)
print(circ.draw())

# Please provide a mitigation plan. The following example is a polynomial fitting ZNE protocol.

def ZNE_Poly_protocol(circ):
    f_value = []
    scale_factor_list = [1,1.01,1.02,1.03]
    for scale_factor in scale_factor_list:
        f_value.append(count_dict_bin2expect(sampling_circuit(circ,scale_factor = scale_factor)))
        # f_value.append(sampling_circuit(circ,scale_factor = scale_factor))

    degree = 1
    print("f_value",f_value)
    coefficients = np.polyfit(np.array(scale_factor_list), np.array(f_value), degree)
    
    # polynomial = np.poly1d(coefficients)
    return np.polyval(coefficients,0)



# benchmark
np.random.seed(2233)
sample_total = 0
N = 100
params_list = [np.random.rand(qubit_num*depth_num)*2*np.pi for index in range(N)]
QEM_val = []
noise_val = []
noise_free_val = []
for index in range(N):
    circ = QGAN_circuit(params_list[index],depth_num =depth_num)
    QEM_val.append(ZNE_Poly_protocol(circ))
    noise_val.append(count_dict_bin2expect(sampling_circuit(circ,scale_factor = 1)))
    # noise_val.append(sampling_circuit(circ,scale_factor = 1))
    noise_free_val.append(circ.expectation_ps(z=[0]))
    print(QEM_val[-1])
    print(noise_val[-1])
    print(noise_free_val[-1])
    # print(ZNE_Poly_protocol(circ))
    # print(circ.expectation_ps(z=[0]))
    # print(count_dict_bin2expect(circ.sample(1000,format = "count_dict_bin")))




    

2024-07-27 21:08:51.294797: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-27 21:08:52.124993: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 21:08:52.125155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 21:08:52.267803: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-27 21:08:52.575239: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-27 21:08:52.579450: I tensorflow/core/platform/cpu_feature_guard.cc:1

      ┌────────────┐   ┌────────────┐                           ┌─────────────┐»
q_0: ─┤ Ry(2.2233) ├─■─┤ Ry(2.2094) ├───────────────────■───────┤ Ry(0.51031) ├»
      ├────────────┤ │ └────────────┘┌───────────┐      │       └─────────────┘»
q_1: ─┤ Ry(4.0135) ├─■───────■───────┤ Ry(5.822) ├──────■──────────────■───────»
      ├────────────┤         │       └───────────┘┌────────────┐       │       »
q_2: ─┤ Ry(4.8607) ├─────────■─────────────■──────┤ Ry(3.0842) ├───────■───────»
     ┌┴────────────┤                       │      ├────────────┤               »
q_3: ┤ Ry(0.79372) ├───────────────────────■──────┤ Ry(4.9769) ├───────────────»
     └─────────────┘                              └────────────┘               »
«                                      
«q_0: ───────────────────■─────────────
«     ┌───────────┐      │             
«q_1: ┤ Ry(3.828) ├──────■────────■────
«     └───────────┘┌────────────┐ │    
«q_2: ──────■──────┤ Ry(1.6214) ├─■──■─
«           │      ├───────────

KeyboardInterrupt: 

In [7]:
print("shot number: ",sample_total)
print("MSE of noise circuit: ",np.mean((np.array(noise_val)-np.array(noise_free_val))**2))
print("MSE of QEM circuit: ",np.mean((np.array(QEM_val)-np.array(noise_free_val))**2))

shot number:  5000000
MSE of noise circuit:  (0.00136151+0j)
MSE of QEM circuit:  (9.286824431856844e-07+0j)
